In [1]:
import pandas as pd
import MalekFinance as mf
import numpy as np
import datetime as dt
from pandas.tseries.offsets import MonthEnd
import statsmodels.api as sm
from tqdm.notebook import tqdm
pd.set_option('display.max_rows', 10)
pd.set_option('display.max_columns', 14)
pd.set_option('display.float_format', '{:.4f}'.format)

In [2]:
Size = mf.read('WRDS Data','All Exchanges CRSP PEMRMNO Monthly Size DataFrame 63-22',index_col=0,parse_dates=True)
RET = mf.read('WRDS Data','All Exchanges CRSP PEMRMNO Monthly Returns DataFrame 63-22',index_col=0,parse_dates=True)
formation_dates = pd.date_range('1967-12-31','2022-11-30',freq='M')

In [3]:
def STR_VW(formation):
    Merged = pd.concat([RET.loc[formation + MonthEnd(1)],Size.loc[formation],RET.loc[formation]],axis=1).dropna()
    Merged.columns = ['Forward Return','Size','Prev Ret']
    Low = Merged[Merged['Prev Ret'] <= Merged['Prev Ret'].quantile(0.2)]
    MidLow = Merged[(Merged['Prev Ret'] >= Merged['Prev Ret'].quantile(0.2)) & (Merged['Prev Ret'] <= Merged['Prev Ret'].quantile(0.4))]
    Mid = Merged[(Merged['Prev Ret'] >= Merged['Prev Ret'].quantile(0.4)) & (Merged['Prev Ret'] <= Merged['Prev Ret'].quantile(0.6))]
    MidHigh = Merged[(Merged['Prev Ret'] >= Merged['Prev Ret'].quantile(0.6)) & (Merged['Prev Ret'] <= Merged['Prev Ret'].quantile(0.8))]
    High = Merged[(Merged['Prev Ret'] >= Merged['Prev Ret'].quantile(0.8)) & (Merged['Prev Ret'] <= Merged['Prev Ret'].quantile(1))]
    for i in Low,MidLow,Mid,MidHigh,High:
        i['Weight'] = i['Size']/sum(i['Size'])
    Low_RET = sum(Low['Forward Return']*Low['Weight'])
    MidLow_RET = sum(MidLow['Forward Return']*MidLow['Weight'])
    Mid_RET = sum(Mid['Forward Return']*Mid['Weight'])
    MidHigh_RET = sum(MidHigh['Forward Return']*MidHigh['Weight'])
    High_RET = sum(High['Forward Return']*High['Weight'])
    list1 = [Low_RET,MidLow_RET,Mid_RET,MidHigh_RET,High_RET]
    Portfolios = pd.DataFrame(list1).transpose()
    Portfolios['DateTime'] = formation + MonthEnd(1)
    Portfolios.set_index(['DateTime'],inplace=True)
    return Portfolios
PortfoliosdfVW = pd.DataFrame()
for i in tqdm(formation_dates):
    PortfoliosdfVW = pd.concat([PortfoliosdfVW,STR_VW(i)])
PortfoliosdfVW.columns = ['Low Reversal','Mid Low Reversal','Mid Reversal','Mid High Reversal','High Reversal']
PortfoliosdfVW['L/S Portfolio'] = mf.long_short(PortfoliosdfVW)
PortfoliosdfVW

  0%|          | 0/660 [00:00<?, ?it/s]

,Low Reversal,Mid Low Reversal,Mid Reversal,Mid High Reversal,High Reversal,L/S Portfolio
DateTime,,,,,,
1968-01-31,-0.0294,-0.0204,-0.0546,-0.0540,-0.0414,0.0121
1968-02-29,-0.0390,-0.0297,-0.0294,-0.0288,-0.0631,0.0241
1968-03-31,0.0133,0.0196,0.0034,0.0075,-0.0029,0.0162
1968-04-30,0.1348,0.0791,0.0885,0.0888,0.1122,0.0226
1968-05-31,0.0183,0.0197,0.0375,0.0249,0.0732,-0.0549
...,...,...,...,...,...,...
2022-08-31,-0.0259,-0.0216,-0.0338,-0.0373,-0.0455,0.0196
2022-09-30,-0.1247,-0.0878,-0.0939,-0.0799,-0.0893,-0.0354
2022-10-31,0.0790,0.0753,0.0767,0.0731,0.0884,-0.0094


In [4]:
def STR_EW(formation):
    Merged = pd.concat([RET.loc[formation + MonthEnd(1)],Size.loc[formation],RET.loc[formation]],axis=1).dropna()
    Merged.columns = ['Forward Return','Size','Prev Ret']
    Low = Merged[Merged['Prev Ret'] <= Merged['Prev Ret'].quantile(0.2)]
    MidLow = Merged[(Merged['Prev Ret'] >= Merged['Prev Ret'].quantile(0.2)) & (Merged['Prev Ret'] <= Merged['Prev Ret'].quantile(0.4))]
    Mid = Merged[(Merged['Prev Ret'] >= Merged['Prev Ret'].quantile(0.4)) & (Merged['Prev Ret'] <= Merged['Prev Ret'].quantile(0.6))]
    MidHigh = Merged[(Merged['Prev Ret'] >= Merged['Prev Ret'].quantile(0.6)) & (Merged['Prev Ret'] <= Merged['Prev Ret'].quantile(0.8))]
    High = Merged[(Merged['Prev Ret'] >= Merged['Prev Ret'].quantile(0.8)) & (Merged['Prev Ret'] <= Merged['Prev Ret'].quantile(1))]
    for i in Low,MidLow,Mid,MidHigh,High:
        i['Weight'] = 1/len(i)
    Low_RET = sum(Low['Forward Return']*Low['Weight'])
    MidLow_RET = sum(MidLow['Forward Return']*MidLow['Weight'])
    Mid_RET = sum(Mid['Forward Return']*Mid['Weight'])
    MidHigh_RET = sum(MidHigh['Forward Return']*MidHigh['Weight'])
    High_RET = sum(High['Forward Return']*High['Weight'])
    list1 = [Low_RET,MidLow_RET,Mid_RET,MidHigh_RET,High_RET]
    Portfolios = pd.DataFrame(list1).transpose()
    Portfolios['DateTime'] = formation + MonthEnd(1)
    Portfolios.set_index(['DateTime'],inplace=True)
    return Portfolios
PortfoliosdfEW = pd.DataFrame()
for i in tqdm(formation_dates):
    PortfoliosdfEW = pd.concat([PortfoliosdfEW,STR_EW(i)])
PortfoliosdfEW.columns = ['Low Reversal','Mid Low Reversal','Mid Reversal','Mid High Reversal','High Reversal']
PortfoliosdfEW['L/S Portfolio'] = mf.long_short(PortfoliosdfEW)
PortfoliosdfEW

  0%|          | 0/660 [00:00<?, ?it/s]

,Low Reversal,Mid Low Reversal,Mid Reversal,Mid High Reversal,High Reversal,L/S Portfolio
DateTime,,,,,,
1968-01-31,0.0529,0.0263,0.0147,0.0175,0.0123,0.0406
1968-02-29,-0.0673,-0.0489,-0.0437,-0.0453,-0.0749,0.0076
1968-03-31,-0.0169,-0.0094,-0.0025,-0.0144,-0.0128,-0.0042
1968-04-30,0.1747,0.1305,0.1239,0.1309,0.1543,0.0204
1968-05-31,0.0855,0.0774,0.0889,0.0885,0.1163,-0.0307
...,...,...,...,...,...,...
2022-08-31,0.0167,-0.0002,-0.0181,-0.0216,-0.0142,0.0309
2022-09-30,-0.1474,-0.1152,-0.0823,-0.0624,-0.1245,-0.0230
2022-10-31,0.0363,0.0568,0.0728,0.0633,0.0250,0.0113


In [5]:
mf.summary_df(PortfoliosdfVW)

,Low Reversal,Mid Low Reversal,Mid Reversal,Mid High Reversal,High Reversal,L/S Portfolio
Annual Return,10.6700,13.0100,11.1900,10.1200,8.3800,2.2900
Annual Volity,24.0500,17.9600,15.4300,15.3200,18.1200,17.0100
Sharpe Ratio,0.4400,0.7200,0.7300,0.6600,0.4600,0.1300
Max Drawdown,-73.8200,-50.8600,-46.2600,-52.5400,-63.1500,-68.3800
Portfolio Beta,1.3500,1.0900,0.9700,0.9500,1.0300,0.3200


In [6]:
mf.summary_df(PortfoliosdfEW)

,Low Reversal,Mid Low Reversal,Mid Reversal,Mid High Reversal,High Reversal,L/S Portfolio
Annual Return,20.9900,12.5300,12.2600,11.2600,4.7800,16.2100
Annual Volity,29.3300,20.1300,17.1500,17.0000,21.1600,17.5100
Sharpe Ratio,0.7200,0.6200,0.7100,0.6600,0.2300,0.9300
Max Drawdown,-62.2100,-57.3200,-59.0300,-66.6400,-82.9600,-45.6600
Portfolio Beta,1.3500,1.0500,0.9200,0.9200,1.0400,0.3100
